In [ ]:
import os
import pandas as pd
import torch
import joblib
import numpy as np
from PIL import Image
from torchvision import transforms, models

# ========== 1. Load CSV and Models ==========
csv_path = '../data/data/test_ids.csv'             # 🔁 Change if needed
test_folder = '../data/data/test'               # 🔁 Folder with test images
output_csv_path = 'predictions_of_IncepSVM.csv'                   # 🔁 Output CSV

df = pd.read_csv(csv_path)
svm = joblib.load('../trained_model/svm_model.joblib')
scaler = joblib.load('../trained_model/scaler.joblib')

# ========== 2. InceptionV3 Model ==========
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inception = models.inception_v3(pretrained=True)
inception.fc = torch.nn.Identity()
inception.to(device).eval()

def extract_features(img_tensor):
    with torch.no_grad():
        features = inception(img_tensor)
    return features.cpu().numpy().flatten()

# ========== 3. Image Preprocessing ==========
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ========== 4. Predict and Append Labels ==========
labels = []

for img_id in df['image_id']:
    img_path = os.path.join(test_folder, img_id)
    if not os.path.exists(img_path):
        print(f"⚠️ Missing image: {img_id}")
        labels.append(-1)  # Or skip / handle as needed
        continue

    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)

    features = extract_features(img_tensor)
    features_scaled = scaler.transform([features])
    
    prediction = svm.predict(features_scaled)[0]  # +1 or -1
    label = 1 if prediction == 1 else 0           # Soil = 1, Non-soil = 0
    labels.append(label)

# ========== 5. Save to New CSV ==========
df['label'] = labels
df.to_csv(output_csv_path, index=False)
print(f"✅ Saved predictions to {output_csv_path}")
